<a href="https://www.kaggle.com/code/jedidiahangekouakou/mysecondfirstnotebook?scriptVersionId=148459500" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# My (Second) First Notebook:
Oh well, I tried to do a simple notebook to identify different characters from the strawhats. But it failed, and got so convoluted that I figured I might as well clean up everything and start from scratch. So here goes.

The process to create a deep-learning model for images seems to be roughly the following
1. Find images to train the model on (obviously this will vary based on the problem)
2. Validate the images and label them somehow. (One way is to simply arrange them in folders)
3. If you got a lot of money and data, train your own model from scratch with super powerful computers **OR** If you're everyone else, use a pretrained model and fine-tune it for your purposes.
4. Test the model
5. If it works, enjoy as this model can be used like pretty much any other standard function. If not starts debugging time. And maybe reconsider if AI is the best solution for your issue.

In [ ]:
# Step 1 : Find images
Since I am really building this as a mean to test the technology, I will simply nerd out. I want to create a model which can precisely identify characters from One Piece, 
more specifically characters from the Mugiwaras. 

As of (2023-10-23), this means:
- Luffy (Captain)
- Zoro (Vice-Captain)
- Nami (Navigator)
- Usopp (Sharpshooter)
- Sanji (Cook)
- Chopper (Doctor)
- Robin (Mom)
- Franky (Shipwright)
- Brook (Musician)
- Jinbei (Helmsman)



# 0. Install what needs to be installed.

In [ ]:
!pip install duckduckgo_search

# 1. Download and Verify the Images

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *
import os

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')
def verify_downloaded_images(directory):
    
    for filename in directory:
        img_path = Path(os.path.join(directory, filename))
        verify(img_path)

In [ ]:
from fastai.vision.all import *
from fastcore.all import *
import os
#We will search all the elements of this list. To make them distinctive, monikers were added.
theMugiwaras = "Monkey D Luffy", "Roronoa Zoro", "Nami, the cat burglar", "Usopp", "Sanji Vinsmoke", "Nico Robin", "Tony Tony Chopper", "Brook, the Soul King", "Franky, Cyborg", "Jinbei"
parent_directory = Path("theMugiwaras")
parent_directory.mkdir(exist_ok=True, parents=True)
for member in theMugiwaras:
    member_directory = Path(os.path.join(parent_directory,member))
    member_directory.mkdir(exist_ok=True, parents=True)
    download_images(member_directory,urls=search_images(member, 50))
    download_images(member_directory,urls=search_images(member + " angry", 50))
    download_images(member_directory,urls=search_images(member + " post timeskip", 50))
    
    

Then after we're done downloading, we use the fastai provided methods to quickly get rid of any image invalid images.

In [ ]:
invalid_images = verify_images(get_image_files(parent_directory))
invalid_images.map(Path.unlink)
len(invalid_images)

# 2. We train the model

First we create a dataloader for it.

In [ ]:
#Create a dataloader
dataloader = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(128, min_scale=0.3),
    batch_tfms=aug_transforms(mult=2)
).dataloaders(parent_directory, bs=32)

dataloader.show_batch(max_n=20, unique=True)

And then we make a learner, learn on our dataset.

In [ ]:
learn = vision_learner(dataloader, resnet18, metrics=error_rate)
learn.fine_tune(10)

# Test
It seems, our learner didn't learn super well. After all it's loss is considerable even after 10 epochs. Barely better than a coin toss. Still for fun, let's try passing a random file to it.

In [ ]:
print(learn.predict("/kaggle/working/theMugiwaras/Monkey D Luffy/e96954a9-8afa-4c23-8a79-0e64d18b778c.png"))
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('/kaggle/working/theMugiwaras/Monkey D Luffy/e96954a9-8afa-4c23-8a79-0e64d18b778c.png')
imgplot = plt.imshow(img)
plt.show()

The above is not too readable, but math tells me that it thinks this image of luffy is actually the 4th person in our list (Usopp). So you've disappointed me AI. Is there anything we can do to help our model get better? Well let's try to clean up the data. Now that we got a working model, we can use the model to sort the data and we can then correct false associations.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

Note. Sadly, we will need to change platforms since Kaggle doesn't show the ImageClassifierCleaner widget. Hence we will need to change platforms. Likely in a proper jupyter setup.

In [1]:
from fastai.vision.widgets import *

cleaner = ImageClassifierCleaner(learn)
cleaner

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


NameError: name 'learn' is not defined